# Fx Option Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-fx) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [1]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Computing the market values and greeks of an OTC Fx Option

This sample shows how to define a vanilla Fx option and compute:
- the market values in domestic and foreign currency (or Deal cuurency) : "MarketValueInDomesticCcy","MarketValueInForeignCcy","MarketValueInDealCcy"
- the market value in report currency : "MarketValueInReportCcy"
- the implied volatility: "ImpliedVolatilityPercent"
- the Greeks in percent: "DeltaPercent","GammaPercent","VegaPercent","ThetaPercent","RhoPercent","VannaPercent","VolgaPercent"
- the delta in domestic and foreign currency: "DeltaAmountInForeignCcy","DeltaAmountInDomesticCcy"

The list of available parameters to define the option and the list of fields that can be retireved or computed is availalbe in [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-fx)

In [2]:
request_body = {
    "fields": ["InstrumentCode","ExerciseType","OptionType","ExerciseStyle","EndDate","StrikePrice","FxSpot","ForwardPoint",\
               "MarketValueInDomesticCcy","MarketValueInForeignCcy","MarketValueInDealCcy","MarketValueInReportCcy","ImpliedVolatilityPercent",\
               "DeltaPercent","GammaPercent","VegaPercent","ThetaPercent","RhoPercent","VannaPercent","VolgaPercent",\
               "DeltaAmountInForeignCcy","DeltaAmountInDomesticCcy","ErrorMessage"],
   
    "universe":[    
        {
            "instrumentType": "Option",
            "instrumentDefinition": {
                "instrumentTag": "Vanilla-Fx-Option",
                "underlyingType": "Fx",
                "underlyingDefinition":{
                    "fxCrossCode":"EURUSD"
                },
                "endDate": "2020-05-20",
                "callPut": "Call",
                "buySell": "Buy",
                "exerciseStyle": "Euro",
                "strike": 1.1229,
                "notionalCcy": "EUR",
                "notionalAmount": 1000000
            },
            
            "pricingParameters": {
                "valuationDate": "2019-11-21",
                "pricingModelType": "BlackScholes",
                "reportCcy":"USD"
            }
        }],
    
    "outputs": ["Data","Headers"]
}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentCode'}, {'type': 'String', 'name': 'ExerciseType'}, {'type': 'String', 'name': 'OptionType'}, {'type': 'String', 'name': 'ExerciseStyle'}, {'type': 'DateTime', 'name': 'EndDate'}, {'type': 'Float', 'name': 'StrikePrice'}, {'type': 'Float', 'name': 'FxSpot'}, {'type': 'Float', 'name': 'ForwardPoint'}, {'type': 'Float', 'name': 'MarketValueInDomesticCcy'}, {'type': 'Float', 'name': 'MarketValueInForeignCcy'}, {'type': 'Float', 'name': 'MarketValueInDealCcy'}, {'type': 'Float', 'name': 'MarketValueInReportCcy'}, {'type': 'Float', 'name': 'ImpliedVolatilityPercent'}, {'type': 'Float', 'name': 'DeltaPercent'}, {'type': 'Float', 'name': 'GammaPercent'}, {'type': 'Float', 'name': 'VegaPercent'}, {'type': 'Float', 'name': 'ThetaPercent'}, {'type': 'Float', 'name': 'RhoPercent'}, {'type': 'Float', 'name': 'VannaPercent'}, {'type': 'Float', 'name': 'VolgaPercent'}, {'type': 'Float', 'name': 'DeltaAmountInForeignCcy'}, {'type': 'Float', 'name': '

In [4]:
import pandas as pd
headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df.T

,0
InstrumentCode,None
ExerciseType,CALL
OptionType,Vanilla
ExerciseStyle,EURO
EndDate,2020-05-20T00:00:00Z
StrikePrice,1.1229
FxSpot,1.1059
ForwardPoint,131.03
MarketValueInDomesticCcy,14005.854177
MarketValueInForeignCcy,12664.666043


### Fx Barrier Options

this samples is similar to the previous one and show how to define a barrier. Double barrier an digital options (Digital, One Touch, No Touch, Double No Touch) are also supported. Documentation is available in [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-fx)

In [5]:
request_body = {
    "fields": ["InstrumentCode","ExerciseType","OptionType","ExerciseStyle","EndDate","StrikePrice","FxSpot",\
               "MarketValueInDomesticCcy","MarketValueInForeignCcy","ImpliedVolatilityPercent",\
               "DeltaPercent","GammaPercent","VegaPercent","ThetaPercent","RhoPercent","VannaPercent","VolgaPercent",\
               "DeltaAmountInforeignCcy","DeltaAmountInDomesticCcy","ErrorMessage"],
   
    "universe":[    
        {
            "instrumentType": "Option",
            "instrumentDefinition": {
                "instrumentTag": "Fx-Barrier-Option",
                "underlyingType": "Fx",
                "underlyingDefinition":{
                    "fxCrossCode":"EURUSD"
                },
                "BarrierDefinition":{
                    "barrierMode": "American",
                    "upOrDown": "Down",
                    "inOrOut": "Out",
                    "level": 1.06
                },
                "endDate": "2020-05-20",
                "callPut": "Call",
                "buySell": "Buy",
                "exerciseStyle": "Euro",
                "strike": 1.1229,
                "notionalCcy": "EUR",
                "notionalAmount": 1000000
            },
            
            "pricingParameters": {
                "valuationDate": "2020-04-16",
                "pricingModelType": "BlackScholes"
            }
        }],
    
    "outputs": ["Data","Headers"]
}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentCode'}, {'type': 'String', 'name': 'ExerciseType'}, {'type': 'String', 'name': 'OptionType'}, {'type': 'String', 'name': 'ExerciseStyle'}, {'type': 'DateTime', 'name': 'EndDate'}, {'type': 'Float', 'name': 'StrikePrice'}, {'type': 'Float', 'name': 'FxSpot'}, {'type': 'Float', 'name': 'MarketValueInDomesticCcy'}, {'type': 'Float', 'name': 'MarketValueInForeignCcy'}, {'type': 'Float', 'name': 'ImpliedVolatilityPercent'}, {'type': 'Float', 'name': 'DeltaPercent'}, {'type': 'Float', 'name': 'GammaPercent'}, {'type': 'Float', 'name': 'VegaPercent'}, {'type': 'Float', 'name': 'ThetaPercent'}, {'type': 'Float', 'name': 'RhoPercent'}, {'type': 'Float', 'name': 'VannaPercent'}, {'type': 'Float', 'name': 'VolgaPercent'}, {'type': 'InvalidField', 'name': 'DeltaAmountInforeignCcy'}, {'type': 'Float', 'name': 'DeltaAmountInDomesticCcy'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [[None, 'CALL', 'Barrier', 'EURO', '2020-05-20T00:00:00Z', 

In [6]:
import pandas as pd
headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df.T

,0
InstrumentCode,None
ExerciseType,CALL
OptionType,Barrier
ExerciseStyle,EURO
EndDate,2020-05-20T00:00:00Z
StrikePrice,1.1229
FxSpot,1.0837
MarketValueInDomesticCcy,1426.445556
MarketValueInForeignCcy,1316.273467
ImpliedVolatilityPercent,8.92556
